In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import utils
import os
import time
import pygaps


mpl.use('TkAgg')
plt.rcParams.update({'font.size': 14})
#plt.rcParams.update({'font.family': 'Times New Roman'})

class Generator:
    def __init__(self):
        dataframe_sorb = pd.read_excel('Silica-loc-isoth1.xlsx', header=None, sheet_name="Adsorption")
        self.data_sorb = dataframe_sorb.to_numpy()
        dataframe_desorb = pd.read_excel('Silica-loc-isoth1.xlsx', header=None, sheet_name="Desorption")
        self.data_desorb = dataframe_desorb.to_numpy()

        self.pressure_start_index = 21
        self.pore_sizes_s = self.data_sorb[0][1:]
        self.pressures_s = self.data_sorb[:, 0][self.pressure_start_index:]
        self.pore_sizes_d = self.data_desorb[0][1:]
        self.pressures_d = self.data_desorb[:, 0][self.pressure_start_index:]

        self.N = self.data_sorb.shape[0]
        self.P_N = len(self.pressures_s)
        self.n_s = np.zeros(len(self.pressures_s))
        self.n_d = np.zeros(len(self.pressures_d))

        self.picture = None
        self.pore_distribution = None

    def generate_pore_distribution(self, sigma1, sigma2, d0_1, d0_2):
        self.pore_distribution = (1 / sigma1) * np.exp(- np.power((self.pore_sizes_d - d0_1), 2) / (2 * sigma1 ** 2))
        self.pore_distribution += (1 / sigma2) * np.exp(- np.power((self.pore_sizes_d - d0_2), 2) / (2 * sigma2 ** 2))
    
    def generate_pore_distribution_for_all_pressure(self, sigma1, sigma2, d0_1, d0_2):
        with open('data/new_a_array_s.npy', 'rb') as f:
            new_a_array_s = np.load(f)
        with open('data/new_a_array_d.npy', 'rb') as f:
            new_a_array_d = np.load(f)
        self.pore_sizes_s = new_a_array_s
        self.pore_distribution_s = (1 / sigma1) * np.exp(- np.power((new_a_array_s - d0_1), 2) / (2 * sigma1 ** 2))
        self.pore_distribution_s += (1 / sigma2) * np.exp(- np.power((new_a_array_s - d0_2), 2) / (2 * sigma2 ** 2))
        
        self.pore_sizes_d = new_a_array_d
        self.pore_distribution_d = (1 / sigma1) * np.exp(- np.power((new_a_array_d - d0_1), 2) / (2 * sigma1 ** 2))
        self.pore_distribution_d += (1 / sigma2) * np.exp(- np.power((new_a_array_d - d0_2), 2) / (2 * sigma2 ** 2))

    def calculate_isotherms(self):
        for p_i in range(self.pressure_start_index, self.data_sorb.shape[0]):
            for d_i in range(len(self.pore_distribution)):
                self.n_s[p_i - self.pressure_start_index] += self.pore_distribution[d_i] * self.data_sorb[p_i][d_i]

        for p_i in range(self.pressure_start_index, self.data_sorb.shape[0]):
            for d_i in range(len(self.pore_distribution)):
                self.n_d[p_i - self.pressure_start_index] += self.pore_distribution[d_i] * self.data_desorb[p_i][d_i]
    
    def calculate_isotherms_from_new_kernel(self):
        with open('data/new_isotherms_s.npy', 'rb') as f:
            data_sorb = np.load(f)
        with open('data/new_isotherms_d.npy', 'rb') as f:
            data_desorb = np.load(f)
        
        for p_i in range(len(self.pressures_s)):
            for d_i in range(len(self.pore_distribution_s)):
                self.n_s[p_i] += self.pore_distribution_s[d_i] * data_sorb[d_i][p_i]

        for p_i in range(len(self.pressures_d)):
            for d_i in range(len(self.pore_distribution_d)):
                self.n_d[p_i] += self.pore_distribution_d[d_i] * data_desorb[d_i][p_i]
    

    def smooth_out_points(self):
        for i in range(1, self.P_N - 1):
            self.n_s[i] = (self.n_s[i - 1] + self.n_s[i] + self.n_s[i + 1]) / 3
            self.n_d[i] = (self.n_d[i - 1] + self.n_d[i] + self.n_d[i + 1]) / 3

    def normalize_data(self):
        max_value = max(self.n_s.max(), self.n_d.max())
        self.n_s = self.n_s / self.n_s.max()
        self.n_d = self.n_d / self.n_d.max()

    def plot_data(self):
        plt.plot(self.pressures_d, self.n_d, marker=".")
        plt.plot(self.pressures_d, self.n_s, marker=".")
        plt.show()

    def plot_distribution(self):
        plt.plot(self.pore_sizes_d, self.pore_distribution, marker=".")
        plt.show()

    def generate_picture(self, resolution):
        self.picture = np.zeros((resolution, resolution), dtype=np.bool_)
        utils.graph_to_picture(self.n_s, self.pressures_s, resolution, self.picture)
        utils.graph_to_picture(self.n_d, self.pressures_d, resolution, self.picture)

    def plot_picture(self):
        fig = plt.figure
        plt.imshow(self.picture, cmap='gray', origin='lower')
        plt.show()
    
    def plot_isotherm(self):
        fig, axs = plt.subplots(2, 1)
        axs[0].plot(self.pressures_s, self.n_s, marker=".", label="Сорбция")
        axs[0].plot(self.pressures_d, self.n_d, marker=".", label="Десорбция")
        axs[0].set_xlabel("Давление")
        axs[0].set_ylabel("Величина адсорбции")
        #axs[1].plot(self.pore_sizes_d, self.pore_distribution_d, marker="*")
        axs[1].plot(self.pore_sizes_s, self.pore_distribution_s, marker=".")
        axs[1].set_ylabel("Функция распределения")
        axs[1].set_xlabel("Размер пор (нм)")
        axs[0].legend()
        plt.show()
    
    def ani(self):
        import numpy as np
        import matplotlib.pyplot as plt
        import matplotlib.animation as animation

        fig, axs = plt.subplots(2, 1, figsize=(8,6))
        gen.generate_pore_distribution_for_all_pressure(sigma1=2, sigma2=3, d0_1=20, d0_2=5)
        gen.calculate_isotherms_from_new_kernel()
        gen.normalize_data()
        sorb_line, = axs[0].plot(self.pressures_s, self.n_s, marker=".", label="Сорбция")
        desorb_line, = axs[0].plot(self.pressures_d, self.n_d, marker=".", label="Десорбция")
        distr_line, = axs[1].plot(self.pore_sizes_s, self.pore_distribution_s, marker=".")

        def animate(i):
            gen.generate_pore_distribution_for_all_pressure(sigma1=2, sigma2=3, d0_1=i, d0_2=5)
            gen.calculate_isotherms_from_new_kernel()
            gen.normalize_data()
            sorb_line.set_ydata(self.n_s)  # update the data
            desorb_line.set_ydata(self.n_d)  # update the data
            distr_line.set_ydata(self.pore_distribution_s)  # update the data
            return sorb_line, desorb_line, distr_line
        
    
        anim = animation.FuncAnimation(fig, animate, np.linspace(5, 70, 300),
                                      interval=25, blit=False)
        writervideo = animation.FFMpegWriter(fps=30) 
        anim.save("anim.mp4", writer=writervideo)
        plt.show()

    def save_picture(self, params, pic_folder, params_folder, ID):
        if not os.path.exists(pic_folder):
            os.makedirs(pic_folder)
        if not os.path.exists(params_folder):
            os.makedirs(params_folder)
        np.save(os.path.join(pic_folder, ID), self.picture)
        np.save(os.path.join(params_folder, ID), params)


    def generate_data_set(sub_folder, base_folder="data"):
        gen = Generator()
        d0_1_range = np.linspace(1, 10, 100)
        d0_2_range = np.linspace(25, 60, 100)
        sigma1 = 3
        sigma2 = 3

        pictures = []
        params = []
        for d0_1 in d0_1_range:
            for d0_2 in d0_2_range:
                gen.generate_pore_distribution(sigma1=sigma1, sigma2=sigma2, d0_1=d0_1, d0_2=d0_2)
                gen.calculate_isotherms()
                gen.smooth_out_points()
                gen.normalize_data()
                gen.generate_picture(resolution=100)
                #gen.save_picture([sigma1, sigma2, d0_1, d0_2], f"data/{sub_folder}/pic", f"data/{sub_folder}/params", ID=f"{ID}")
                pictures.append(gen.picture)
                params.append([sigma1, sigma2, d0_1, d0_2])

        path = f"{base_folder}/{sub_folder}"
        if not os.path.exists(path):
            os.makedirs(path)
        np.save(f"{path}/{sub_folder}_x", pictures)
        np.save(f"{path}/{sub_folder}_y", params)


    def generate_simple_data_set(self):
        sigma1 = 3
        sigma2 = 3
        d0_1_range = np.linspace(1, 10, 15)
        d0_2_range = np.linspace(25, 65, 15)
        i = 0
        for d0_1 in d0_1_range:
                for d0_2 in d0_2_range:
                    i+=1
                    gen.generate_pore_distribution_for_all_pressure(sigma1=sigma1, sigma2=sigma2, d0_1=d0_1, d0_2=d0_2)
                    gen.calculate_isotherms_from_new_kernel()
                    gen.normalize_data()
                    
                    
                    point_isotherm = pygaps.PointIsotherm(
                    # First the pandas.DataFrame
                    isotherm_data=pd.DataFrame({
                        'pressure' : np.concatenate((self.pressures_s, self.pressures_d), axis = 0),             # required
                        'loading' :  np.concatenate(( gen.n_s, gen.n_d), axis = 0),              # required
                    }),
                        material='carbon',              # Required
                        adsorbate='nitrogen',           # Required
                        temperature=77, 
                        
                        loading_key='loading',          # The loading column
                        pressure_key='pressure',        # The pressure column
                        d0_1 = d0_1,
                        d0_2 = d0_2,
                        
                    )
                    
                    point_isotherm.to_aif(f"test_set/{i}.aif")
                    #np.savez(f"test_set/{i}", d0_1 = d0_1, d0_2 = d0_2, n_s = gen.n_s, n_d = gen.n_d)


if __name__ == "__main__":
    #generate_data_set(sub_folder="test")
    gen = Generator()
    
    #gen.generate_pore_distribution_for_all_pressure(sigma1=2, sigma2=3, d0_1=20, d0_2=5)
    #gen.calculate_isotherms_from_new_kernel()
    #gen.normalize_data()
    #gen.plot_isotherm()
    
    gen.generate_simple_data_set()
    
    #gen.ani()
    #gen.smooth_out_points()
    # gen.normalize_data()
    # gen.generate_picture(100)
    # gen.save_picture([2, 5, 9, 20], "data/test/pic", "data/test/params", ID="TEST.txt")


In [ ]:
# plt.plot(gen.pressures_d, norm, marker=".")
# plt.show()

In [3]:
with open('new_data/new_a_array_s.npy', 'rb') as f:
    new_a_array_s = np.load(f)
with open('new_data/new_a_array_d.npy', 'rb') as f:
    new_a_array_d = np.load(f)

with open('new_data/new_isotherms_s.npy', 'rb') as f:
    data_sorb = np.load(f)
with open('new_data/new_isotherms_d.npy', 'rb') as f:
    data_desorb = np.load(f)

In [75]:
df1 = pd.DataFrame(data_sorb,
                   columns = gen.pressures_s,
                   index=new_a_array_s).T

df2 = pd.DataFrame(data_desorb,
                   columns = gen.pressures_s,
                   index=new_a_array_s).T


with pd.ExcelWriter('newT.xlsx') as writer:  
    df1.to_excel(writer, sheet_name='sorb')
    df2.to_excel(writer, sheet_name='desorb')

In [22]:
def get_k_b(x1, y1, x2, y2):
        k = (y1 - y2) / (x1 - x2)
        b = y1 - k*x1
        return k, b

def find_nearests(a, n):
    value1 = a[np.argsort(np.abs(a-n))[0]]
    index1 = np.where(a == value1)[0]
    if value1 > n:
        index2 = index1 - 1
    else:
        index2 = index1 + 1
    return index1, index2

dataframe_sorb = pd.read_excel('SBA-15.xlsx', header=None, sheet_name="SBA-15")
# plt.plot(exp_p, exp_sorb, marker=".")
# plt.show()
exp_data = dataframe_sorb.to_numpy()
exp_p = exp_data[:, 3][1:75]
exp_sorb = exp_data[:, 4][1:75]
new_exp_sorb = np.empty(gen.pressures_s.shape)
# exp_scale = np.max(new_exp_sorb)
# new_exp_sorb = new_exp_sorb / exp_scale

for i in range(len(gen.pressures_s)):
    index1, index2 = find_nearests(exp_p, gen.pressures_s[i])
    x1 = exp_p[index1]
    x2 = exp_p[index2]
    y1 = exp_sorb[index1]
    y2 = exp_sorb[index2]
    k, b = get_k_b(x1, y1, x2, y2)
    new_exp_sorb[i] = k * gen.pressures_s[i] + b

In [24]:
exp_scale = np.max(new_exp_sorb)
new_exp_sorb = new_exp_sorb / exp_scale

In [26]:
plt.plot(exp_p, exp_sorb / exp_scale, marker=".")
plt.plot(gen.pressures_s, new_exp_sorb, marker=".")
plt.show()

In [7]:
delta_d = new_a_array_s[1:] - new_a_array_s[0:1] 
delta_d[0], new_a_array_s[0], new_a_array_s[1]

(1.1502727342849184e-05, 1.0554515347427873, 1.0554630374701301)

In [66]:
# нормирование
p_len = data_sorb.shape[1]
d_len = data_sorb.shape[0]
norm = np.zeros(p_len)
data_sorb_normed = np.zeros(data_sorb.shape)
nfit = np.zeros(p_len)
f_k = np.ones(d_len)
f_k_pr = np.ones(d_len)


# for k in range(d_len):
#     for i in range(p_len):
#         norm[k] += data_sorb[k][i]

for j in range(p_len):
    for i in range(d_len):
        norm[j] += data_sorb[i][j]


for j in range(d_len):
    for i in range(p_len):
        if norm[i] != 0:
            data_sorb_normed[j][i] = data_sorb[j][i] / norm[i]
        else:
            data_sorb_normed[j][i] = 0        

for i in range(p_len):
    for k in range(d_len):
        nfit[i]=data_sorb_normed[k][i]*f_k[k]


In [62]:
nfit

array([0.00208565, 0.00208565, 0.00182729, 0.00167998, 0.00159824,
       0.00155905, 0.00154893, 0.00155897, 0.00158254, 0.00161381,
       0.00164704, 0.00167531, 0.00169871, 0.00172102, 0.00174405,
       0.00176857, 0.00179187, 0.00181223, 0.00183162, 0.00185094,
       0.00187081, 0.0018922 , 0.00191224, 0.00192885, 0.00194288,
       0.00195952, 0.0019754 , 0.00198859, 0.00200194, 0.00201495,
       0.00203002, 0.00204635, 0.00206339, 0.00208391, 0.00209493,
       0.00210479, 0.00211322, 0.00212114, 0.00212791, 0.00213314,
       0.00213444, 0.00213898, 0.00214413, 0.00214923, 0.00215526,
       0.00216166, 0.00216739, 0.00217296, 0.00217945, 0.00218583,
       0.00219262, 0.00219416, 0.00219665, 0.00219882, 0.00220131,
       0.00220414, 0.00220644, 0.00220947, 0.00221185, 0.00221255,
       0.00221289, 0.00221295, 0.00221443, 0.00221587, 0.00221706,
       0.00221898, 0.00222088, 0.00222274, 0.00222385, 0.00222492,
       0.00222599, 0.00222728, 0.00222843, 0.00222942, 0.00223

In [63]:
for step in range(5):     
        for i in range(p_len):
            sum = 0
            for k in range(d_len - 1):
                sum += data_sorb_normed[k][i]*f_k[k] * delta_d[k]
            nfit[i] = sum

        for j in range(d_len - 1):
            sum = 0
            for i in range(p_len):
                sum += nfit[i] / new_exp_sorb[i] * data_sorb_normed[j][i] * delta_d[k]
            f_k[j] = f_k_pr[j] * sum
            f_k_pr[j] = f_k[j]
        
    
        
        

In [64]:
f_k

array([3.63230189e+119, 1.55103267e+119, 7.55859482e+118, 4.03899891e+118,
       2.31252407e+118, 1.39947018e+118, 8.88050499e+117, 5.88068549e+117,
       4.04997651e+117, 2.88982444e+117, 2.12344776e+117, 1.59852208e+117,
       1.22984029e+117, 9.66139719e+116, 7.74936538e+116, 6.33749611e+116,
       5.27244478e+116, 4.45738417e+116, 3.82736743e+116, 3.33774500e+116,
       2.95764425e+116, 2.65965648e+116, 2.42863415e+116, 2.25392786e+116,
       2.12960157e+116, 2.05073339e+116, 2.01599745e+116, 2.02316682e+116,
       2.07065348e+116, 2.15696765e+116, 2.27971412e+116, 2.43444558e+116,
       1.50469826e+116, 9.65386652e+115, 1.25258782e+116, 1.51812567e+116,
       8.65140752e+115, 5.15479863e+115, 5.33948831e+115, 7.20789272e+115,
       6.62520607e+115, 5.95154064e+115, 5.27686678e+115, 4.98533057e+115,
       4.66465486e+115, 4.46165346e+115, 4.22470796e+115, 4.37616238e+115,
       4.57420545e+115, 4.13736677e+115, 4.15351251e+115, 4.17753114e+115,
       4.21303339e+115, 4

In [67]:
plt.plot(gen.pressures_s, nfit, marker=".")
plt.show()

In [65]:
plt.plot(new_a_array_s, f_k, marker=".")
plt.show()